In [1]:
import datasets
import torch
import numpy as np
import json

## Get negation indices

In [2]:
mnli = datasets.load_dataset("glue", "mnli")

Reusing dataset glue (/home/meissner/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
negation_words = ["no", "not", "don't", "none", "nothing", 
                  "never", "aren’t", "isn’t", "weren’t", 
                  "neither", "don’t", "didn’t", "doesn’t", 
                  "cannot", "hasn’t", "won’t"]

In [4]:
def find_negation_idxs(dataset):
    ent_indices, cont_indices = [], []
    for sample in dataset:
        if any([word in sample["hypothesis"].lower() for word in negation_words]):
            if sample['label'] == 0:
                ent_indices.append(sample['idx'])
            elif sample['label'] == 2:
                cont_indices.append(sample['idx'])
    return ent_indices, cont_indices

In [5]:
train_ents, train_conts = find_negation_idxs(mnli["train"])
val_ents, val_conts = find_negation_idxs(mnli["validation_matched"])

In [6]:
all_indices = {"train_ents" : train_ents, "train_conts" : train_conts, "val_ents" : val_ents, "val_conts" : val_conts}

In [7]:
!mkdir /home/meissner/shortcut-pruning/data/subsets/

mkdir: cannot create directory ‘/home/meissner/shortcut-pruning/data/subsets/’: File exists


In [8]:
with open("/home/meissner/shortcut-pruning/data/subsets/mnli_negation_indices.json", "w") as _file:
    _file.write(json.dumps(all_indices))